<a href="https://colab.research.google.com/github/dajebbar/FreeCodeCamp-python-data-analysis/blob/main/simpleImputer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip install opendatasets --upgrade --quiet

In [40]:
import opendatasets as od
import os

od.version()
url = 'https://www.kaggle.com/jsphyg/weather-dataset-rattle-package'
od.download(url)


data_dir = 'weather-dataset-rattle-package'

os.listdir(data_dir)
train_csv = data_dir + '/weatherAUS.csv'

Skipping, found downloaded files in "./weather-dataset-rattle-package" (use force=True to force download)


In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import missingno as msno

plt.style.use('fivethirtyeight')
%matplotlib inline

In [65]:
raw_df = pd.read_csv(train_csv)

categorical_column = ['Location','WindGustDir', 'WindDir9am', 
                      'WindDir3pm',  'RainToday']
numerical_column = ['MinTemp', 'MaxTemp', 'Sunshine', 'WindSpeed9am', 
                    'Pressure3pm', 'Temp9am', 'Temp3pm', 'Cloud3pm' ]

target_column = 'RainTomorrow'
year = pd.to_datetime(raw_df.Date).dt.year
date = 'Date'

df = raw_df[categorical_column + numerical_column + [target_column]]
df.head()

,Location,WindGustDir,WindDir9am,WindDir3pm,RainToday,MinTemp,MaxTemp,Sunshine,WindSpeed9am,Pressure3pm,Temp9am,Temp3pm,Cloud3pm,RainTomorrow
0,Albury,W,W,WNW,No,13.4,22.9,NaN,20.0,1007.1,16.9,21.8,NaN,No
1,Albury,WNW,NNW,WSW,No,7.4,25.1,NaN,4.0,1007.8,17.2,24.3,NaN,No
2,Albury,WSW,W,WSW,No,12.9,25.7,NaN,19.0,1008.7,21.0,23.2,2.0,No
3,Albury,NE,SE,E,No,9.2,28.0,NaN,11.0,1012.8,18.1,26.5,NaN,No
4,Albury,W,ENE,NW,No,17.5,32.3,NaN,7.0,1006.0,17.8,29.7,8.0,No


## Dealing with Nan's

In [66]:
df[numerical_column].isna().sum()

MinTemp          1485
MaxTemp          1261
Sunshine        69835
WindSpeed9am     1767
Pressure3pm     15028
Temp9am          1767
Temp3pm          3609
Cloud3pm        59358
dtype: int64

In [67]:
df[categorical_column].isna().sum()

Location           0
WindGustDir    10326
WindDir9am     10566
WindDir3pm      4228
RainToday       3261
dtype: int64

In [57]:
df[target_column].isna().any()

True

In [68]:
df2 = df.copy()

In [69]:
df2.dropna(subset=[target_column, 'RainToday'], inplace=True)

In [70]:
df2[target_column].isna().sum()

0

### Imputing Missing Numeric Data

In [61]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
# impute the original dataframe
imputer.fit(df2[numerical_column])

SimpleImputer()

In [62]:
df2[numerical_column].isna().sum()

MinTemp           468
MaxTemp           307
Sunshine        66805
WindSpeed9am     1055
Pressure3pm     13769
Temp9am           656
Temp3pm          2624
Cloud3pm        56094
dtype: int64

In [71]:
df2[numerical_column].sample(15)

,MinTemp,MaxTemp,Sunshine,WindSpeed9am,Pressure3pm,Temp9am,Temp3pm,Cloud3pm
112586,12.3,17.9,NaN,17.0,1013.6,15.1,16.4,NaN
129771,12.1,25.6,NaN,4.0,1018.5,16.0,24.6,NaN
14082,14.2,32.3,11.7,19.0,1013.4,21.8,31.3,1.0
104324,11.7,28.0,11.2,30.0,1016.2,14.4,26.0,0.0
39111,8.0,24.2,10.4,13.0,1027.8,16.0,23.7,1.0
38106,-0.4,12.1,5.3,9.0,1028.2,1.3,11.4,7.0
46807,11.5,19.7,7.1,15.0,1022.5,12.3,17.5,5.0
88484,19.2,24.7,0.0,30.0,1014.4,21.4,23.5,8.0
56876,6.8,18.8,NaN,28.0,1021.5,10.5,16.4,5.0
49221,0.6,14.9,NaN,0.0,1027.8,6.3,14.1,NaN


In [72]:
for col in numerical_column:
  df2[col].fillna(df2[col].mean(), inplace=True)

df2[numerical_column].sample(15)

,MinTemp,MaxTemp,Sunshine,WindSpeed9am,Pressure3pm,Temp9am,Temp3pm,Cloud3pm
35907,17.3,21.7,0.80000,35.0,1022.4,18.0,18.5,8.00000
50512,4.9,21.1,7.63054,19.0,1015.7,11.1,19.1,4.49925
107165,3.5,15.1,8.70000,9.0,1022.5,8.8,14.2,1.00000
145353,22.0,37.4,7.63054,4.0,1007.7,29.4,36.1,5.00000
28378,7.2,17.5,7.63054,15.0,1013.5,13.9,15.8,4.49925
143025,26.7,30.6,7.63054,11.0,1000.1,27.2,24.8,8.00000
134109,9.3,22.0,7.63054,7.0,1016.3,14.9,20.2,4.49925
94971,23.6,26.7,0.30000,20.0,1015.7,24.0,25.5,8.00000
106927,9.0,25.2,12.60000,28.0,1020.8,13.7,23.1,0.00000
11554,20.1,33.0,7.63054,19.0,1002.5,27.9,30.2,4.49925


In [73]:
df2[numerical_column].isna().sum()

MinTemp         0
MaxTemp         0
Sunshine        0
WindSpeed9am    0
Pressure3pm     0
Temp9am         0
Temp3pm         0
Cloud3pm        0
dtype: int64